In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt

from pasna_analysis import DataLoader, Experiment

experiment_name = '20240515'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(DataLoader(exp_path))

In [ ]:
'''Plots activity signals for a group of embryos.'''
# Change start and end to select different groups
start = 0
end = 3
embryos = exp.embryos[start:end]

fig, ax = plt.subplots(end-start, 2, figsize=(12, 4*(end-start)))
for i, emb in enumerate(embryos):
    time = emb.activity[:, 0]
    trace = exp.traces[emb.name]
    ax[i][0].set_title(f'{emb.name} - GCamP')
    ax[i][0].plot(time, trace.active, color='green')
    ax[i][0].axvline(time[trace.trim_idx], color='k', linestyle='--')
    ax[i][1].set_title(f'{emb.name} - tdTomato')
    ax[i][1].plot(time, trace.struct, color='firebrick')
    ax[i][1].axvline(time[trace.trim_idx], color='k', linestyle='--')

In [ ]:
'''Plots dff for a group of embryos.'''
# Change start and end to select different groups
start = 0
end = 3
embryos = exp.embryos[start:end]

fig, ax = plt.subplots(end-start, figsize=(8, 4*(end-start)))
for i, emb in enumerate(embryos):
    time = emb.activity[:, 0]
    trace = exp.traces[emb.name]
    time = time[:trace.trim_idx]
    ax[i].set_title(f'{emb.name} - dff')
    ax[i].plot(time, trace.dff)